### group members
* Max Dunger
* [Yannik Motzet](mailto:yannik.motzet@outlook.com)

# Übung 3: Naiver Bayesklassifikator
## 2. Textklassifikation

In [1]:
import tarfile
import os
import re
import numpy as np

a.) Extract raw data

In [ ]:
tar = tarfile.open("20news-18828.tar.gz", "r:gz")
tar.extractall()
tar.close()

b.) Load Data

In [2]:
relevant_newsgroups = ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
top_directory = r'.\20news-18828'

data = []
data_class = []

for newsgroup in relevant_newsgroups:
    directory = top_directory + '\\' + newsgroup
    
    for file in os.listdir(directory):
        file_path = directory + '//' + file
        f = open(file_path ,"r")
        file_string = f.read()
        data.append(file_string)
        data_class.append(newsgroup)
print('done: ' + str(len(data)) + ' entries')

done: 3387 entries


In [3]:
# delete first two lines
def strip_header(text):
    _before , _blankline , after = text.partition ('\n\n')
    return after
data = [strip_header(text) for text in data]

c.) Token

In [4]:
token = []
for element in data:
    l = re.compile(r"(?u)\b\w\w+\b").findall(element.lower())
    token.extend(l)

features = set(token)
features = list(features)

In [5]:
# design-matrix
data_frequency = np.empty((len(data), len(features)))

for i in range(len(data)):
    for j in range(len(features)):
        data_frequency[i, j] = data[i].count(features[j])

In [6]:
# training / test set
border = int(0.6 * data_frequency.shape[0])

training_data = data_frequency[0:border, :]
training_data_class = data_class[0:border]

validation_data = data_frequency[border+1:, :]
validation_data_class = data_class[border+1:]

d.)  naiven Bayes-Klassifikator

In [7]:
a_priori = []

for newsgroup in relevant_newsgroups:
    number = 0
    for i in range(len(training_data)):
        if training_data_class[i] == newsgroup:
            number += 1
    a_priori.append(number/2032)

Berechnung wie weit Klassen gehen

In [8]:
class_index = []
class_index.append(0)
last_class = training_data_class[0]
index = 0

for element in training_data_class:
    if element != last_class:
        class_index.append(index)
        last_class = element
    index+=1
class_index.append(index)
class_index

[0, 799, 1772, 2032]

Berechnung Anzahl aller Wörter pro Klasse (für Laplace-Glättung)

In [9]:
class_number_words = []

for i in range(1, len(class_index)):
    class_number_words.append(np.sum(training_data[class_index[i-1]:class_index[i]-1, :]))

class_number_words

[1438212.0, 1349409.0, 481919.0]

Berechnung von Likelihood mit Laplace-Glättung

In [10]:
likelihood = np.full((3, len(features)), 1.0)

for i in range(1, len(class_index)):
    for j in range(len(features)):
        likelihood[i-1, j] = (np.sum(training_data[class_index[i-1]:class_index[i]-1, j]) + 1) / (class_number_words[i-1] + len(features))

Wahrscheinlichkeit für Klasse 1 für erster Testdateneintrag

In [12]:
p_class_0 = 1

for i in range(len(validation_data[0, :])):
    if validation_data[0, i] != 0:
        p_class_0 = p_class_0 * (validation_data[0, i] * likelihood[0, i])
p_class_0 = p_class_0 * a_priori[0]

In [13]:
class_index = 0

for k in range(3):
    p_class = 1
    for i in range(len(validation_data[k, :])):
        if validation_data[k, i] != 0:
            p_class = p_class * (validation_data[k, i] * likelihood[class_index, i]) # log
    p_class = p_class * a_priori[class_index]
    print(p_class)

0.0
0.0
0.0


Multonomilal Naive Bayes with sklearn

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_frequency, data_class, test_size=0.4, random_state=0)

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [15]:
import pandas as pd
dataframe= pd.DataFrame(data=clf.predict(X_test), columns=["prediciton"])
prediction = dataframe.to_numpy()

In [16]:
dataframe.insert(0, "test", y_test)

In [17]:
dataframe

,test,prediciton
0,comp.graphics,comp.graphics
1,alt.atheism,alt.atheism
2,alt.atheism,alt.atheism
3,sci.space,sci.space
4,comp.graphics,sci.space
...,...,...
1350,comp.graphics,sci.space
1351,alt.atheism,alt.atheism
1352,sci.space,sci.space
1353,comp.graphics,comp.graphics


In [18]:
accuracy = 0

for i in range(len(y_test)):
    if y_test[i] == prediction[i]:
        accuracy += 1

accuracy = accuracy/len(y_test)

In [19]:
accuracy

0.8797047970479704